# SWELL-KW Scores

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [ ]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
# To include edgeml in python path
sys.path.insert(0, '../../')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_FastGRNN,EMI_FastRNN
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [ ]:
NUM_HIDDEN = 128
NUM_TIMESTEPS = 30
NUM_FEATS = 22
FORGET_BIAS = 1.0
NUM_OUTPUT = 3
USE_DROPOUT = 0

KEEP_PROB = 0.9
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"
WRANK = 5
URANK = 6
PREFETCH_NUM = 5
BATCH_SIZE = 32
NUM_EPOCHS = 50
NUM_ITER = 4
NUM_ROUNDS = 2

# A staging direcory to store models
MODEL_PREFIX = '/home/sf/data/SWELL-KW/FGModels_30_10/'

# Loading Data

In [ ]:
path="/home/sf/data/SWELL-KW/30_10/"
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

In [ ]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True
    
def addExtendedAssignOps(self, graph, W_val=None, B_val=None):
    W1 = graph.get_tensor_by_name('W1:0')
    B1 = graph.get_tensor_by_name('B1:0')
    W1_op = tf.assign(W1, W_val)
    B1_op = tf.assign(B1, B_val)
    self.assignOps.extend([W1_op, B1_op])

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob, **kwargs):
    feedDict = {self._emiGraph.keep_prob: keep_prob}
    return feedDict
    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
EMI_FastGRNN.addExtendedAssignOps = addExtendedAssignOps

def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index


if USE_DROPOUT is True:
    EMI_Driver.feedDictFunc = feedDictFunc

## 1. Initializing a New Computation Graph

In [ ]:
tf.reset_default_graph()

inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                              gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

# Construct the graph
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)


In [ ]:
emiDriver.initializeSession(g1)
#y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
#                                      y_train=y_train, bag_train=BAG_TRAIN,
#                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
#                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
#                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
#                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
#                                      fracEMI=0.5, updatePolicy='top-k', k=1)

In [ ]:
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

emiDriver.initializeSession(g1)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))

## 2. Loading a Saved Graph into EMI-Driver

In [ ]:
tf.reset_default_graph()
emiDriver.initializeSession(g1)
emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX , 1039)
k = 1

In [ ]:
%%time
predictions, predictionStep = emiDriver.getInstancePredictions(x_test[:64], y_test[:64], earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)

In [ ]:
print('Accuracy at k = %d: %f' % (k,  np.mean(int(bagPredictions == BAG_TEST))))

## 3. Initializing using a Saved Graph

In [ ]:
# Making sure the old graph and sessions are closed
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

In [ ]:
tf.reset_default_graph()

sess = tf.Session()
graphManager = edgeml.utils.GraphManager()
graph = graphManager.loadCheckpoint(sess, MODEL_PREFIX, globalStep=1004)

In [ ]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT, graph=graph)
emiLSTM = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy', graph=graph)

g1 = graph
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

In [ ]:
emiDriver.setSession(sess)

In [ ]:
%%time

# tf.reset_default_graph()
# emiDriver.initializeSession(g1)
# emiDriver.loadSavedGraphToNewSession(MODEL_PREFIX, 1007)
k = 1
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                            minProb=0.99, keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))


In [ ]:
x_test.shape, bagPredictions.shape,y_test.shape

## 4. Restoring from Numpy Matrices

In [ ]:
graph = tf.get_default_graph()
W1 = graph.get_tensor_by_name('W1:0')
B1 = graph.get_tensor_by_name('B1:0')
allVars = emiLSTM.varList + [W1, B1]
sess = emiDriver.getCurrentSession()
allVars = sess.run(allVars)

base = '/tmp/models/'
np.save(base + 'kernel.npy', allVars[0])
np.save(base + 'bias.npy', allVars[1])
np.save(base + 'W1.npy', allVars[2])
np.save(base + 'B1.npy', allVars[3])

In [ ]:
sess = emiDriver.getCurrentSession()
sess.close()
tf.reset_default_graph()

In [ ]:
base = '/home/iot/Documents/EdgeML-master/tf/examples/EMI-RNN/GRNN model'
kernel = np.load(base + 'kernel.npy')
bias = np.load(base + 'bias.npy')
W = np.load(base + 'W1.npy')
B = np.load(base + 'B1.npy')

In [ ]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS,
                                 NUM_OUTPUT)
emiLSTM = EMI_Fast(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)
    # Add the assignment operations
    emiLSTM.addBaseAssignOps(graph, [kernel, bias])
    emiLSTM.addExtendedAssignOps(graph, W, B)
    # Setup the driver. You can run the initializations manually as well
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

emiDriver.initializeSession(graph)
# Run the assignment operations
sess = emiDriver.getCurrentSession()
sess.run(emiLSTM.assignOps)

k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test,
                                                               earlyPolicy_minProb,
                                                               minProb=0.99,
                                                               keep_prob=1.0)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k,
                                             numClass=NUM_OUTPUT)
print('PART IV: Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions ==
                                                        BAG_TEST).astype(int))))